In [1]:
import os, sys, time, math
from natsort import natsorted, ns

import numpy as np
import cv2
import matplotlib.pyplot as plt
import cma
from PIL import Image
import glob
import re


from gvxrPython3 import gvxr
from gvxrPython3 import json2gvxr

from gvxrPython3.utils import visualise # Visualise the 3D environment if k3D is supported
from gvxrPython3.utils import plotScreenshot # Visualise the 3D environment using Matplotlib

from gvxrPython3.utils import loadSpekpySpectrum # Generate and load an X-ray spectrum using Spekpy
from gvxrPython3.utils import loadXpecgenSpectrum # Generate and load an X-ray spectrum using xpecgen

spekpy is not install, you won't be able to load a beam spectrum using spekpy
SimpleGVXR 2.0.6 (2023-05-23T20:42:48) [Compiler: Microsoft Visual Studio] on Windows
gVirtualXRay core library (gvxr) 2.0.6 (2023-05-23T20:43:44) [Compiler: Microsoft Visual Studio] on Windows


In [2]:
use_padding = True
pad_width = 50
angular_step_in_deg = 3.6




In [3]:
# data_path = "C:/Users/user/phd/Dataoff/"
# data_path = "C:/Users/user/phd/26SepImages/"
# data_path = "C:/Users/snn23kfl/project/"
#data_path = "4thOCtober_image/"
#data_path = "4thCotober_imageAngle/"
#data_path = "23OctoberImage/"
data_path = "25OctoberData/"


In [4]:
current_folder = str(globals()['_dh'][0])
print(current_folder)

C:\Users\snn23kfl\project


In [5]:
def average_images(image_paths):
    
    """Average a list of images."""
  
    # Load the first image to get the shape
    sample_image = cv2.imread(image_paths[0], cv2.IMREAD_GRAYSCALE)
    if sample_image is None:
        raise ValueError(f"Failed to load image: {image_paths[0]}")
    
    avg_image = np.zeros_like(sample_image, dtype=float)
    
    if use_padding:
        avg_image = np.pad(avg_image, (pad_width, pad_width), mode='median')

    for path in image_paths:
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        
        if image is None:
            raise ValueError(f"Failed to load image: {path}")

        if use_padding:
            median_value = np.median(image)
            image = np.pad(image, (pad_width, pad_width), 'constant', constant_values=(median_value, median_value))

        avg_image += image.astype(float)
    
    avg_image /= len(image_paths)
    
    return cv2.medianBlur(avg_image.astype(np.single), 3)

In [6]:
def flatField(img, white, dark, epsilon=0.0):

    temp_white = np.copy(white)
    temp_img = np.copy(img)
    
    test = white - dark == 0
    temp_white[test] += 1

    if len(img.shape) == 2:
        temp_img[test] += 1
        return (temp_img - dark + epsilon) / (temp_white - dark + epsilon)
    elif len(img.shape) == 3:
        flat = np.zeros(img.shape, dtype=np.single)
        for i, proj in enumerate(temp_img):
            proj[test] += 1
            flat[i] = (proj - dark + epsilon) / (temp_white - dark + epsilon)
        return flat
    else:
        raise IOError("Bad image dimension: " + str(img.shape))

In [7]:
#path for my recently acquired images stored in PhD file



dark_field_paths = glob.glob(data_path + '/darkfd/darkfd_*.tiff')
dark_field_paths = natsorted(dark_field_paths, key=lambda y: y.lower())

white_field_paths = glob.glob(data_path + '/whitefd/whitefd_*.tiff')
white_field_paths = natsorted(white_field_paths, key=lambda y: y.lower())

raw_image_paths = glob.glob(data_path + '/raw_images/raw_image_*.tiff')
raw_image_paths = natsorted(raw_image_paths, key=lambda y: y.lower())

if len(raw_image_paths) == 0:
    raw_image_paths = glob.glob(data_path + '/rawimages/raw_images_*.jpg')
    raw_image_paths = natsorted(raw_image_paths, key=lambda y: y.lower())

In [8]:
#Average the dark field and white field images
I_dark = average_images(dark_field_paths)
I_white = average_images(white_field_paths)

In [9]:
I_raw = []
angles_in_deg = []
for i, fname in enumerate(raw_image_paths):
    angle = angular_step_in_deg * i
    print(f"Processing image {i}: Angle = {angle}")  # Debug print

    if angle < 360.000001:
        angles_in_deg.append(angular_step_in_deg * i)
        image = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Image read failed: {fname}")
        else:
            if use_padding:
                median_value = np.median(image)
                image = np.pad(image, (pad_width, pad_width), 'constant', constant_values=(median_value, median_value))
            I_raw.append(image)

print(f"Number of images after processing: {len(I_raw)}")


In [10]:
def save_images(images, base_path, prefix):
    """
    Saves an array of images to files with incremental indices.

    Args:
    - images (numpy.ndarray): An array of images to save.
    - base_path (str): The base directory where the images will be saved.
    - prefix (str): A prefix for the saved image filenames.

    Returns:
    - None
    """
    # Create the base path if it does not exist
    if not os.path.exists(base_path):
        os.makedirs(base_path)

    # Save each image in the array
    for i, image in enumerate(images):
        filename = os.path.join(base_path, f"{prefix}_{i:04d}.tiff")
        cv2.imwrite(filename, image)

    print(f"All images saved with prefix {prefix} in {base_path}")

In [11]:
corrected_path = os.path.join(data_path, "corrected")

In [12]:
# Save the flat-field corrected images using zeros as dark field
save_images(I_flat2, corrected_path, "corrected_flatfield_zeros")

All images saved with prefix corrected_flatfield_zeros in 25OctoberData/corrected


In [13]:
# This assumes the extra pixels are evenly distributed around the image
crop_x = (I_dark.shape[1] - 640) // 2
crop_y = (I_dark.shape[0] - 480) // 2
I_dark_cropped = I_dark[crop_y:crop_y+480, crop_x:crop_x+640]
I_white_cropped = I_white[crop_y:crop_y+480, crop_x:crop_x+640]


In [14]:
I_flat1 = flatField(I_raw, I_white_cropped, I_dark_cropped)


IndexError: boolean index did not match indexed array along dimension 0; dimension is 580 but corresponding boolean dimension is 480

In [ ]:
# Define the directory to save the flat-field corrected images
flatfield_corrected_dir = 'C:/Users/snn23kfl/flat_field'

# Create the directory if it does not exist
if not os.path.exists(flatfield_corrected_dir):
    os.makedirs(flatfield_corrected_dir)

# Now you can loop through your corrected images and save them
for i, corrected_image in enumerate(I_flat1):
    # Convert from float to 16-bit integer
    corrected_image = cv2.convertScaleAbs(corrected_image * (2**16 - 1), alpha=(2**16 - 1))

    # File name
    image_name = f"flatfield_corrected_1_{i:04d}.tif"

    # Save image
    cv2.imwrite(os.path.join(flatfield_corrected_dir, image_name), corrected_image)
    print(f"Saved flat-field corrected image {image_name}")

print("All flat-field corrected images have been saved.")

In [ ]:
corrected_img = I_flat1[0]  # Assume I_flat1[0] is the first corrected image
print("Before scaling:", corrected_img.min(), corrected_img.max())

scaled_img = cv2.convertScaleAbs(corrected_img * (2**16 - 1), alpha=(2**16 - 1))
print("After scaling:", scaled_img.min(), scaled_img.max())
